In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
import os

### Step 1: Load base model

In [ ]:
from huggingface_hub import login
login(os.getenv('HF_API'))

model_id="meta-llama/Llama-2-13b-chat-hf"
custom_cache_directory = os.getenv('CACHE_DIR')

tokenizer = LlamaTokenizer.from_pretrained(model_id, cache_dir=custom_cache_directory)
model = LlamaForCausalLM.from_pretrained(
    model_id, 
    load_in_8bit=True, 
    device_map='auto', 
    torch_dtype=torch.float16, 
    cache_dir=custom_cache_directory
)

### Step 2: Check base model

In [ ]:
prompt = "Hi how are you doing?"
model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
input_length = model_input.input_ids.size(1)

model.eval()
with torch.no_grad():
    output = model.generate(**model_input, max_new_tokens=100)[0]
    generated_sequence = output[input_length:]  # Extract only the generated tokens
    return_message = tokenizer.decode(generated_sequence, skip_special_tokens=True)
    print(return_message)

### Step 3: Prepare model for PEFT

In [ ]:
model.train()

def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)

### Step 4: Define an optional profiler

In [ ]:
from transformers import TrainerCallback
from contextlib import nullcontext
enable_profiler = False
output_dir = "tmp/llama-output"

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Set up profiler
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
    
    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler
            
        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

### Step 5: Fine Tune the model

In [ ]:
# TODO: get dataset and finetune

from transformers import default_data_collator, Trainer, TrainingArguments



# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )

    # Start training
    trainer.train()

In [ ]:
model.save_pretrained(output_dir)

### Step 6: Revaluate on fine tuned model

In [ ]:
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))